<a href="https://colab.research.google.com/github/kkk020719/DFT_Fall24/blob/main/GradDFT_TrainEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/XanaduAI/GradDFT.git
!pip install torch_geometric
!pip install jax==0.4.20 jaxlib==0.4.20
!pip list | grep jax

  Cloning https://github.com/XanaduAI/GradDFT.git to /tmp/pip-req-build-n3sv9v5o
  Running command git clone --filter=blob:none --quiet https://github.com/XanaduAI/GradDFT.git /tmp/pip-req-build-n3sv9v5o
  Resolved https://github.com/XanaduAI/GradDFT.git to commit 941afc28a6b24477369d638b07ca64c6a46365ba
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want

In [ ]:
#import os

# Mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

# Create a folder in the root directory
#!mkdir -p "/content/drive/My Drive/Ethanol_data_MD17"

In [6]:
from torch_geometric.datasets import md17
import jax
from jax import numpy as jnp
from jax import grad, jit, vmap
from jax import random
import math
import pdb
import numpy as np

from grad_dft import (
	energy_predictor,
	simple_energy_loss,
	NeuralFunctional,
	molecule_from_pyscf,
  Functional
)

from pyscf import gto, dft

from jax.nn import sigmoid, gelu
from jax.random import PRNGKey
from jax import value_and_grad
from flax import linen as nn
from optax import adam, apply_updates
from tqdm import tqdm
from jax.flatten_util import ravel_pytree
import matplotlib.pyplot as plt

from grad_dft import constraints

from sklearn.model_selection import train_test_split

In [ ]:
ethanol_dataset = md17.MD17(root='.', name='ethanol')

In [4]:

print("Energy of the Molecule:")
print(ethanol_data.energy / 627.509)  # This will print the energy of the molecule.

Cartesian Positions of Atoms:
tensor([[ 2.1540e-02, -5.5601e-01,  7.0714e-04],
        [-1.2741e+00,  2.4780e-01,  5.0275e-03],
        [ 1.1265e+00,  3.1391e-01, -3.5790e-03],
        [ 5.9397e-02, -1.2191e+00,  8.8888e-01],
        [-8.4126e-02, -1.1607e+00, -8.8107e-01],
        [-1.4003e+00,  8.4873e-01,  8.7578e-01],
        [-1.3232e+00,  8.5038e-01, -9.2045e-01],
        [-2.1540e+00, -4.2816e-01, -2.1951e-02],
        [ 1.9455e+00, -2.0287e-01,  4.7284e-02]])
Atomic Numbers:
tensor([6, 6, 8, 1, 1, 1, 1, 1, 1])
<built-in method item of Tensor object at 0x7e955d8632e0>
Energy of the Molecule:
tensor([-154.9091])


In [15]:
def obtain_data(num_data, data_set, atomic_symbols):
  'Takes a number of molecules randomly from dataset provided and returning the PySCF molecule objects'
  molecules = []
  #Randomly select a number of indices
  indices = np.random.choice(len(data_set), num_data, replace=False)
  selected_samples = [data_set[i] for i in indices]
  # Define atomic symbols for the corresponding atomic numbers in MD17 dataset for ethanol
  for sample in selected_samples:
    position = sample.pos
    atomic_number = sample.z

    # Start building the molecule description string
    atom_desc = ''
    for z, pos in zip(atomic_number, position):
      #print(z.item())
      #print(pos)
      if z.item() in atomic_symbols:
        symbol = atomic_symbols[z.item()]  # Get symbol using atomic number
        atom_desc += f'{symbol} {pos[0]:.4f} {pos[1]:.4f} {pos[2]:.4f}\n'
      else:
        raise ValueError(f"Unrecognized atomic number {z.item()} found, unable to proceed with molecule creation.")
    # Create a PySCF molecule object
    mol = gto.M(atom=atom_desc, basis='def2-svp', unit='Ang', spin = 0, verbose=0)
    molecules.append(mol)

  return molecules

#loading PySCF data
ethanol_atomic_symbols = {1: 'H', 6: 'C', 8: 'O'}
molecule_data = obtain_data(5, ethanol_dataset, ethanol_atomic_symbols)

6
tensor([ 0.3317, -0.1047, -0.4598])
6
tensor([-0.6103,  1.2270, -0.2505])
8
tensor([ 0.3106, -1.0113,  0.6711])
1
tensor([ 0.0316, -0.7746, -1.3801])
1
tensor([ 1.2824,  0.3131, -0.5983])
1
tensor([-0.6176,  1.6228, -1.2530])
1
tensor([-1.6455,  1.0740,  0.0527])
1
tensor([-0.1555,  1.8640,  0.5676])
1
tensor([-0.5076, -1.4226,  0.4243])
6
tensor([ 0.2285, -0.2704, -0.5515])
6
tensor([-1.2001,  0.0409, -0.1163])
8
tensor([0.8722, 0.2228, 0.6161])
1
tensor([ 0.2030, -1.3997, -0.7283])
1
tensor([ 0.4694,  0.3404, -1.4034])
1
tensor([-1.6763,  1.0278, -0.3847])
1
tensor([-1.8741, -0.5672, -0.7293])
1
tensor([-1.2751, -0.3966,  0.9044])
1
tensor([1.8848, 0.1920, 0.5194])
6
tensor([ 0.3055, -0.0737, -0.3709])
6
tensor([-0.5883,  1.2250, -0.2782])
8
tensor([ 0.3266, -1.0564,  0.6191])
1
tensor([-0.0695, -0.5220, -1.3428])
1
tensor([ 1.3176,  0.2321, -0.5746])
1
tensor([-0.6452,  1.6442, -1.3135])
1
tensor([-1.5745,  0.7881, -0.0924])
1
tensor([-0.2758,  2.0419,  0.3108])
1
tensor([-0.5680,

# Training using GradDFT and direct optimization

In this notebook, we will run through the different strategies that can be used to train a Neural functional for the exchange-correlation functional in Grad DFT. Using molecules such as ethanol as training molecules, we will study how a neural functional can generalize to calculating the total energy.



In [ ]:
import jax
from jax import numpy as jnp
from jax import grad, jit, vmap
from jax import random
import math
import pdb

from grad_dft import (
	energy_predictor,
	simple_energy_loss,
	NeuralFunctional,
	molecule_from_pyscf,
  Functional
)

from pyscf import gto, dft

from jax.nn import sigmoid, gelu
from jax.random import PRNGKey
from jax import value_and_grad
from flax import linen as nn
from optax import adam, apply_updates
from tqdm import tqdm
from jax.flatten_util import ravel_pytree
import matplotlib.pyplot as plt

from grad_dft import constraints

from sklearn.model_selection import train_test_split

## Generating training/validation data and initial guesses

The data were acuiqred from MD17 dataset. According to their documentation, the energies of molecules of MD17 dataset were calculated at the PBE/def2-SVP level of theory using very tight SCF convergence and very dense DFT integration grid.

In [ ]:
#all_molecules = []
train_molecules = []
test_molecules = []
ground_energies = []

def generate_ethanol_variants(n_molecules):
    # Base coordinates for ethanol
    seed = 40
    molecules = []
    base_coords = [
        "C 0.0000 0.0000 0.0000", "C 0.0000 0.0000 1.5400",
        "O 0.0000 0.0000 -1.4300", "H 0.0000 0.9300 -1.9300",
        "H 0.0000 -0.9300 -1.9300", "H 0.0000 0.9300 2.0700",
        "H 0.0000 -0.9300 2.0700", "H 0.9200 0.0000 1.9300",
        "H -0.9200 0.0000 1.9300"
    ]

    return molecules

# Generate molecules
n_total = 5  # Total number of molecules
ethanol_molecules = generate_ethanol_variants(n_total)

In [ ]:
# Splitting the data into training and testing sets
train_molecules, test_molecules = train_test_split(ethanol_molecules, test_size=0.20, random_state=42)
train_ground_energies, test_ground_energies = train_test_split(ground_energies, test_size=0.20, random_state=42)

# Print the sizes of the resulting sets
print(f"Training set size: {len(train_molecules)}")
print(f"Testing set size: {len(test_molecules)}")

# Train

In [ ]:
def energy_densities(molecule):
    rho = molecule.density()
    lda_e = -3/2 * (3/(4*jnp.pi))**(1/3) * (rho**(4/3)).sum(axis = 1, keepdims = True) #
    #lda_e = jnp.concatenate([lda_e, lda_e], axis=0)
    return lda_e

def coefficient_inputs(molecule):
    rho = molecule.density()
    kinetic = molecule.kinetic_density()
    return jnp.concatenate((rho, kinetic), axis=1)

def coefficients(self, rhoinputs):
    x = nn.Dense(features=2)(rhoinputs) # features = 1 means it outputs a single weight
    #print(x)
    #pdb.set_trace()
    x = nn.LayerNorm()(x)
    #print("coe")
    #print(x)
    #print(gelu(x))
    return gelu(x) # using gelu as activation function

neuralfunctional = NeuralFunctional(coefficients, energy_densities, coefficient_inputs)

In [ ]:
seed = 40
key = jax.random.PRNGKey(seed)
params_train = []
cinputs = [coefficient_inputs(molecule) for molecule in train_molecules]
for i in cinputs:
  params_train.append(neuralfunctional.init(key, i))
#predicted_energy = neuralfunctional.energy(params, molecule)

# Training

In [ ]:
# Define optimizer
n_epochs, learning_rate, momentum = 200, 1e-2, 0.9
optimizer = adam(learning_rate=learning_rate, b1=momentum)
opt_state = optimizer.init(params)
compute_energy = energy_predictor(neuralfunctional)
trueenergy = mf.energy_tot()
converging_energies = []

@value_and_grad
def SEloss(params, compute_energy, molecule, trueenergy):
    predictedenergy, fock_matrix = compute_energy(params, molecule) #returns both the xc and fock matrix
    converging_energies.append(predictedenergy)
    return (predictedenergy - trueenergy) ** 2 #sqaured error used

# Testing
